In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 2
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.06 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.01
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_5S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 2.0170998573303223 Train_acc: 23.437500%
Train Epoch: 2/90 Train_Loss: 1.4208314418792725 Train_acc: 44.062500%
Train Epoch: 3/90 Train_Loss: 1.0823315382003784 Train_acc: 64.375000%
Train Epoch: 4/90 Train_Loss: 0.6115437746047974 Train_acc: 67.812500%
Train Epoch: 5/90 Train_Loss: 0.5649047493934631 Train_acc: 81.250000%
Train Epoch: 6/90 Train_Loss: 0.3538503348827362 Train_acc: 86.250000%
Train Epoch: 7/90 Train_Loss: 0.43824994564056396 Train_acc: 90.937500%
Train Epoch: 8/90 Train_Loss: 0.3414873480796814 Train_acc: 93.437500%
Train Epoch: 9/90 Train_Loss: 0.1321665197610855 Train_acc: 95.312500%
Train Epoch: 10/90 Train_Loss: 0.17127874493598938 Train_acc: 98.437500%
Train Epoch: 11/90 Train_Loss: 0.14996

Train Epoch: 13/90 Train_Loss: 0.11293256282806396 Train_acc: 97.812500%
Train Epoch: 14/90 Train_Loss: 0.0674014613032341 Train_acc: 97.812500%
Train Epoch: 15/90 Train_Loss: 0.04972381144762039 Train_acc: 99.062500%
Train Epoch: 16/90 Train_Loss: 0.02480434440076351 Train_acc: 99.375000%
Train Epoch: 17/90 Train_Loss: 0.01799515075981617 Train_acc: 98.750000%
Train Epoch: 18/90 Train_Loss: 0.024301055818796158 Train_acc: 100.000000%
Train Epoch: 19/90 Train_Loss: 0.007994168438017368 Train_acc: 99.062500%
Train Epoch: 20/90 Train_Loss: 0.03362276405096054 Train_acc: 98.437500%
Train Epoch: 21/90 Train_Loss: 0.028639327734708786 Train_acc: 99.375000%
Train Epoch: 22/90 Train_Loss: 0.15648725628852844 Train_acc: 98.125000%
Train Epoch: 23/90 Train_Loss: 0.013161417096853256 Train_acc: 99.687500%
Train Epoch: 24/90 Train_Loss: 0.03970528393983841 Train_acc: 98.125000%
Train Epoch: 25/90 Train_Loss: 0.029808759689331055 Train_acc: 100.000000%
Train Epoch: 26/90 Train_Loss: 0.005007859785

Train Epoch: 28/90 Train_Loss: 0.008417061530053616 Train_acc: 99.375000%
Train Epoch: 29/90 Train_Loss: 0.027762705460190773 Train_acc: 99.687500%
Train Epoch: 30/90 Train_Loss: 0.00983651727437973 Train_acc: 99.687500%
Train Epoch: 31/90 Train_Loss: 0.026669763028621674 Train_acc: 99.687500%
Train Epoch: 32/90 Train_Loss: 0.004769604653120041 Train_acc: 100.000000%
Train Epoch: 33/90 Train_Loss: 0.009899675846099854 Train_acc: 100.000000%
Train Epoch: 34/90 Train_Loss: 0.020250603556632996 Train_acc: 100.000000%
Train Epoch: 35/90 Train_Loss: 0.006992754526436329 Train_acc: 99.687500%
Train Epoch: 36/90 Train_Loss: 0.006449558772146702 Train_acc: 100.000000%
Train Epoch: 37/90 Train_Loss: 0.004742036573588848 Train_acc: 100.000000%
Train Epoch: 38/90 Train_Loss: 0.008733459748327732 Train_acc: 100.000000%
Train Epoch: 39/90 Train_Loss: 0.006745860446244478 Train_acc: 100.000000%
Train Epoch: 40/90 Train_Loss: 0.004594889003783464 Train_acc: 100.000000%
Train Epoch: 41/90 Train_Loss: 

Train Epoch: 43/90 Train_Loss: 0.00247816345654428 Train_acc: 100.000000%
Train Epoch: 44/90 Train_Loss: 0.0018759120721369982 Train_acc: 100.000000%
Train Epoch: 45/90 Train_Loss: 0.002181611256673932 Train_acc: 100.000000%
Train Epoch: 46/90 Train_Loss: 0.0024457653053104877 Train_acc: 100.000000%
Train Epoch: 47/90 Train_Loss: 0.003330047009512782 Train_acc: 100.000000%
Train Epoch: 48/90 Train_Loss: 0.0017895731143653393 Train_acc: 100.000000%
Train Epoch: 49/90 Train_Loss: 0.0019184742122888565 Train_acc: 100.000000%
Train Epoch: 50/90 Train_Loss: 0.0013801086461171508 Train_acc: 100.000000%
Train Epoch: 51/90 Train_Loss: 0.0015207264805212617 Train_acc: 100.000000%
Train Epoch: 52/90 Train_Loss: 0.0030560242012143135 Train_acc: 100.000000%
Train Epoch: 53/90 Train_Loss: 0.0025461085606366396 Train_acc: 100.000000%
Train Epoch: 54/90 Train_Loss: 0.0019694925285875797 Train_acc: 100.000000%
Train Epoch: 55/90 Train_Loss: 0.0017846139380708337 Train_acc: 100.000000%
Train Epoch: 56/

Train Epoch: 58/90 Train_Loss: 0.002651316812261939 Train_acc: 100.000000%
Train Epoch: 59/90 Train_Loss: 0.0024579104501754045 Train_acc: 100.000000%
Train Epoch: 60/90 Train_Loss: 0.0028171201702207327 Train_acc: 100.000000%
Train Epoch: 61/90 Train_Loss: 0.00352739030495286 Train_acc: 100.000000%
Train Epoch: 62/90 Train_Loss: 0.002857272047549486 Train_acc: 100.000000%
Train Epoch: 63/90 Train_Loss: 0.013978035189211369 Train_acc: 100.000000%
Train Epoch: 64/90 Train_Loss: 0.0021287843119353056 Train_acc: 100.000000%
Train Epoch: 65/90 Train_Loss: 0.0023264093324542046 Train_acc: 100.000000%
Train Epoch: 66/90 Train_Loss: 0.0024034962989389896 Train_acc: 100.000000%
Train Epoch: 67/90 Train_Loss: 0.004974652547389269 Train_acc: 100.000000%
Train Epoch: 68/90 Train_Loss: 0.00425952672958374 Train_acc: 100.000000%
Train Epoch: 69/90 Train_Loss: 0.0014942401321604848 Train_acc: 100.000000%
Train Epoch: 70/90 Train_Loss: 0.00271974946372211 Train_acc: 100.000000%
Train Epoch: 71/90 Tra

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()